# Configurar o ambiente



Necessário apenas instalar a biblioteca biopython com PIP (porque o python já vem por padrão neste servidor).

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.9 MB/s eta 0:00:00


# Exercício 1

**Objetivo**: reconhecer operações essenciais de sequência (tamanho, slicing, GC, transcrição, tradução).

Considere o código abaixo como base e exemplo para os exercícios a seguir:

In [1]:
from Bio.Seq import Seq
seq = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print("len:", len(seq))
print("primeiros 10 nt:", seq[:10])
print("A:", seq.count("A"), "C:", seq.count("C"), "G:", seq.count("G"), "T:", seq.count("T"))

revcomp = seq.reverse_complement()
print("revcomp:", revcomp)

rna = seq.transcribe()
prot = seq.translate(to_stop=True)
print("RNA:", rna)
print("Prot:", prot)

len: 39
primeiros 10 nt: ATGGCCATTG
A: 9 C: 8 G: 14 T: 8
revcomp: CTATCGGGCACCCTTTCAGCGGCCCATTACAATGGCCAT
RNA: AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
Prot: MAIVMGR


1a: Escreva uma função gc(seq) que retorne %GC com 2 casas.

In [6]:
def gc(seq):
    totalC = seq.count("C")
    totalG = seq.count("G")
    return round((totalG + totalC)/len(seq) * 100,2)
x = gc(seq)
x

56.41

1b: Agora teste em seq e em revcomp (o %GC muda? por quê?)

In [7]:
y = gc(revcomp)
y

56.41

Nao, pois revcomp eh o par complementar de seq, e como o par de C eh G, nada muda

1c (para casa): Verifique se seq contém o motivo GGG (use o operador in) e retorne as posições onde ocorre

In [9]:
motif = "GGG"
positions = []
start = 0
while True:
    pos = seq.find(motif, start)
    if pos == -1:
        break
    positions.append(pos)
    start = pos + 1
print(positions)

[14, 26]


# Exercício 2

**Objetivo**: fazer parsing de FASTA, sumarizar, filtrar e gravar resultados.

Considere o código abaixo como base e exemplo para os exercícios a seguir:

In [10]:
from Bio import SeqIO

def gc(seq):
    s = str(seq).upper()
    return 100*(s.count("G")+s.count("C"))/len(s)

in_fa = "Genomas_filoviridae.fasta"  # troque pelo seu caminho

n = 0
tamanhos, gcs = [], []
for rec in SeqIO.parse(in_fa, "fasta"):
    n += 1
    tamanhos.append(len(rec.seq))
    gcs.append(gc(rec.seq))
print(f"lidas {n} sequências")
print("tamanho médio:", sum(tamanhos)/len(tamanhos))
print("GC médio:", sum(gcs)/len(gcs))

lidas 833 sequências
tamanho médio: 18917.597839135655
GC médio: 40.93000407855796


2a: Pesquisando em um livro de virologia, você descobriu que o genoma completo de Filovirus deve ser sempre maior do que 18700 nucleotídeos. Filtre e salve em `filtered.fasta` apenas as sequências de genoma completo de Filoviridae. Quantas sequências foram removidas por não serem genomas completos?

In [11]:
filtered_fasta = []
count = 0
for rec in SeqIO.parse(in_fa, "fasta"):
    if len(rec.seq) >= 18700:
        filtered_fasta.append(rec)
        count += 1
print(f"sequências filtradas: {n-count}")
SeqIO.write(filtered_fasta, "filtered.fasta", "fasta")

sequências filtradas: 18


815

2b: Crie um CSV com id, length, gc para todas as sequências.

In [15]:
import pandas as pd


rows = []
for rec in SeqIO.parse("filtered.fasta", "fasta"):
        rows.append({
        "id": rec.id,
        "length": len(rec.seq),
        "gc": round(gc(rec.seq), 2)  
    })
df = pd.DataFrame(rows)
with open("filtered.csv", "w") as f:
    f.write(df.to_csv(index=False))
    

# Exercício 3

**Objetivo**: Ler FASTQ, acessar qualidades Phrede filtrar de forma simples as sequências.

Considere o código abaixo como base e exemplo para os exercícios a seguir:

In [16]:
from Bio import SeqIO
import statistics as stats

in_fq = "Exemplo_sequencias_R1.fastq"  # troque pelo seu caminho

qual_medias = []
comprimentos = []
for rec in SeqIO.parse(in_fq, "fastq"):  # assume FASTQ Sanger (Phred+33)
    q = rec.letter_annotations["phred_quality"]
    qual_medias.append(sum(q)/len(q))
    comprimentos.append(len(rec))
print("reads:", len(qual_medias),
      "| média das médias de Q:", round(sum(qual_medias)/len(qual_medias), 2),
      "| tamanho médio:", round(sum(comprimentos)/len(comprimentos), 1))

reads: 110185 | média das médias de Q: 33.42 | tamanho médio: 115.1


3a: Grave em `highq.fastq` apenas as reads com média de Q ≥ 25 e len ≥ 75.

In [ ]:
with open("highq.fastq", "w") as f:
    for rec in SeqIO.parse(in_fq, "fastq"):
        q = rec.letter_annotations["phred_quality"]
        if sum(q)/len(q) >= 25 and len(rec) <= 75:
            f.write(rec)
        

TypeError: write() argument must be str, not SeqRecord

3b (Desafio): Pegue todas as sequências do arquivo FASTQ e calcule a probabilidade média de erro (convertida a partir da fórmula do Phred) para cada posição da sequência.  
Por exemplo, qual a chance de erro na posição 1 de todas as sequências? E na posição 2? Assim por diante.  
Faça um histrograma da probabilidade média de erro por posição da sequência para identificar algum padrão (Quem identificar, me conta depois ;)